# A tutorial for: building deploying a model on Amazon Sagemaker

Deployment refers to the act of making your machine learning model available in a production environment, where it can be accessed and utilised by other tools, workflows and software. Deployment is typically one of the last stages in the machine learning workflow and can be one of the most difficult.

This is where [Amazon SageMaker](https://aws.amazon.com/sagemaker/) comes in. SageMaker is a relatively new Amazon service that supports all of the steps of a machine learning model development: data labelling, model building, training, optimization, and deployment. You can choose to use all or any combination of these key features in SageMaker. In addition, SageMaker is based on Jupyter notebooks (which are familiar to most data scientists these days), comes with many built-in state-of-the-art algorithms, and provides a host of ready-to-use examples to get you up and running quickly.

In this tutorial I'll walk you through building and deploying a machine learning model using SageMaker. While the SageMaker docs are quite good, I still found it a little difficult to get going on SageMaker - so this tutorial provides a simple walkthrough of using SageMaker for model buidling and deployment.

We'll be using SageMaker’s implementation of the XGBoost algorithm to train and host a regression model to predict the age of abalone based on the classic abalone dataset hosted [here](https://archive.ics.uci.edu/ml/datasets/abalone). We aim to predict the age of abalone based on eight physical measurements. The data provided in the tutorial has been modified from the original, namely, the "Sex" characteristic has been one-hot-encoded and the "rings" target variable has had +1.5 added to it to represent the abalone age in years. The data is located in the [data folder](./data). It has been shuffled, one-hot-encoded and split into training and validation sets for you. The training set has 3341 rows and the validation set has 836 rows. The data looks like this:

<img src="docs/img/sagemaker_images/sm_0.png" width="600">

If you're interested in other ways of deploying your machine learning model, check out [my other tutorial using Flask](deploy-with-flask.ipynb).

# Contents

1. [Requirements](#1)
2. [Preparing the data](#2)
3. [Setting up SageMaker](#3)
4. [Building and deploying the model](#4)
5. [Using the model](#5)
6. [End](#6)

# 1. Requirements <a id=1></a>

To complete this tutorial, you'll need the following:

- An AWS account. Register [here](https://console.aws.amazon.com/).
- The Postman app and a free account. Download/register [here](https://www.postman.com/).

# 2. Preparing the data <a id=2></a>

Many of the Amazon SageMaker examples provided in the docs use data that are downloaded from online sources, but I wanted to emulate the process of using a custom dataset in this tutorial. To make the data available to SageMaker we need to host it in an S3 bucket:

1. Head over to AWS, log in, and search for S3:

<img src="docs/img/sagemaker_images/sm_1.png" width="600">

2. Choose "Create Bucket".
3. Provide a globally unique name for your bucket. I named mine "deploy-tutorial-tb".
4. Leave remaining settings as default and click "Create" in the lower left. You've now created a storage bucket to hold your data.

<img src="docs/img/sagemaker_images/sm_2.png" width="600">

5. Click on the bucket you just created and then click "Upload" on the top left corner.
6. Drag and drop into the upload prompt the two data files provided with this repository, called "abalone_train.csv" and "abalone_validation.csv". Then click "Upload" in the lower left corner to complete the upload.

<img src="docs/img/sagemaker_images/sm_3.png" width="600">

7. You will see the data now in the bucket. Your data is now ready to be accessed by SageMaker.

<img src="docs/img/sagemaker_images/sm_4.png" width="600">

# 3. Setting up SageMaker <a id=3></a>

We will now prepare SageMaker to build and deploy a machine learning model.

1. Head back to the main AWS dashboard and search for SageMaker.

<img src="docs/img/sagemaker_images/sm_5.png" width="600">

2. Click "Notebook instances" in the panel on the left side of the screen. Then click "Create notebook instance" in top right of the screen.

<img src="docs/img/sagemaker_images/sm_6.png" width="600">

3. Give your notebook instance a name, I called mine "deploy-tutorial-tb".
4. Scroll down and in the IAM role field select "Create a new role" from the drop down. In the pop-up, select "Any S3 bucket" and then click "Create role" at the bottom right (feel free to specify only a specific bucket, the one you created previously, if you wish).

<img src="docs/img/sagemaker_images/sm_7.png" width="600">

5. Leave all remaining fields as default, scroll to the bottom of the page and click "Create notebook instance".
6. Wait a few minutes for status of your newly created notebook to change from "Pending" to "InService".

<img src="docs/img/sagemaker_images/sm_8.png" width="600">

7. Click "Open Jupyter" to open your notebook instance.

We'll now upload the notebook I've created for this tutorial, which is located in the [deploy-with-sagemaker folder](deploy-with-sagemaker) in this repository

1. Click the "New" dropdown button at the top right and select "Folder". Click the checkbox next to your newly created folder, and then click "Rename" in the menu bar above to give the folder a name such as "deploy-tutorial".

2. Click the folder to enter it and then click "Upload" in top right corner. Choose the [xgboost_abalone.ipynb](deploy-with-sagemaker/xgboost_abalone.ipynb) notebook file downloaded from this repo and upload it. Open up the notebook. We'll use this notebook to build and deploy the model as described in the following section. At this point, your screen should look something like the following:

<img src="docs/img/sagemaker_images/sm_9.png" width="600">

> Note 1: If you get an error "Kernel not found". Use the dropdown menu that appeared to choose the "conda_python3" kernel and select "Set kernel".

> Note 2: Amazon SageMaker has many example notebooks available for you to use - no matter what your use case is, you should be able to find a base notebook to work off. You can check out the available examples by clicking the "SageMaker Examples" tab in your opened notebook instance (note that it may take a few minutes for the examples to become available to you, you can periodically click the little refresh button at the top right corner of the screen to refresh the list). You can either "Preview" (just to have a look) or "Use" (will copy the notebook to your root) these notebooks.

# 4. Building and deploying the model <a id=4></a>

In this section we will build and deploy a model to predict the age of abalone. The notebook we just uploaded has all the details of the dataset and XGBoost model we'll be using for this task. Let's walk through it step-by-step:

1. We first need to point our notebook to the dataset we uploaded to S3 previously. All that is required here is to enter the name of your S3 bucket into the cell and run it (recall that I called mine "deploy-tutorial-tb").

<img src="docs/img/sagemaker_images/sm_10.png" width="600">

2. SageMaker uses Docker containers to allow users to train and deploy models. There are many pre-built Docker images available, particularly suited for SageMaker's built-in algorithms and we will use one of those here (note however that you can always upload your own custom docker container - more on that in the [docs](https://docs.aws.amazon.com/sagemaker/latest/dg/your-algorithms.html)). Run the next cell to load the xgboost docker container.

<img src="docs/img/sagemaker_images/sm_11.png" width="600">

3. The following cell sets the parameters for, and executes training of the XGBoost model. You shouldn't need to change anything here, but feel free to take a look at what's going on inside this cell. The cell will periodically print feedback on the status of the training job. When it's finished you should see a "Completed" message - training should take around 5 minutes.

<img src="docs/img/sagemaker_images/sm_12.png" width="600">

4. Now we need to create a SageMaker model from the training job above. Run cells under the headings "4. Create the model" and "5. Create endpoint" to create the model and deploy it to an endpoint that will be available to provide inferences - it will take about 10 minutes to run all of these cells (the notebook provides more details on what each of these cells are doing). You will eventually receive an output that your endpoint has been created.
5. If you stop the tutorial here, be sure to shut down the endpoint you created by running the cell under the heading "7. Delete Endpoint" - else you will continue to be charged by Amazon.

> Note that a key functionality of SageMaker is model tuning. I'm not describing any tuning/optimization here, but it is well described in the [SageMaker docs](https://docs.aws.amazon.com/sagemaker/latest/dg/automatic-model-tuning.html)

# 5. Using the model <a id=5></a>

Now that we've deployed the model as an endpoint, we need want to use it in a production environment. There are two main ways we may want to use our model:

1. To act as a HTTPS endpoint that can provide inferences on a case-by-case basis (e.g., for a web application). We will focus on this use case here.
2. To get predictions for an entire dataset. You can read about this use case in the [SageMaker docs](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-batch.html).


To allow users to interact with our machine learning model we need an API. Put simply, APIs facilitate applications to work together without having to know exactly how they’re implemented. You can think of a simple wall plug as an API: manufactures of electronic equipment know that they can attach a particular power cord to their device which will allow the device to "interface with" (i.e., use) the electricity supply through the wall plug - the manufacturers don't need to know how the electricity is supplied or what kinf of infrastructure is behing the wall plug, they just know that if their device fits the wall plug, they can access the electricity. We will use a combination of Amazon API Gateway and Amazon Lambda to create an API that will allow users to use our machine learning model for predictions. This section is a shortened summary of this [Amazon blog post](https://aws.amazon.com/blogs/machine-learning/call-an-amazon-sagemaker-model-endpoint-using-amazon-api-gateway-and-aws-lambda/).

### 5.1 Create a Lambda function that calls the SageMaker Runtime Invoke_Endpoint

Our first task is to create a function that will be invoked through an API request. We'll create that funciton using Amazon Lambda.

1. Head back to the main AWS dashboard and search for Lambda.

<img src="docs/img/sagemaker_images/sm_13.png" width="600">

2. In the Lambda dashboard select "Create function".
3. With the "Author from scratch" tab selected, give your function a name (I called the function "predict_abalone"), choose Python 3.6, and select "Create a new role with basic Lambda permissions".

<img src="docs/img/sagemaker_images/sm_14.png" width="600">

4. Before we write any code, select the "Permissions" tab at the top of the screen, and then click on the new role that was created for you when you created the lambda function.

<img src="docs/img/sagemaker_images/sm_15.png" width="600">

5. In the new screen that opens up, click the name of the role called e.g., "AWSLambdaBasicExecutionRole-\*", then click "Edit policy", click the JSON tab, and then replace the contents of the JSON with the following. Then click "Review policy" at the bottom right, and then "Save changes".

```
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "VisualEditor0",
            "Effect": "Allow",
            "Action": "sagemaker:InvokeEndpoint",
            "Resource": "*"
        }
    ]
}
```

<img src="docs/img/sagemaker_images/sm_16.png" width="600">

6. Now head back to Amazon Lambda and click the "Configuration" tab. Scroll down and copy-and-paste the following code into the editor. This is the function we will trigger with an API request.

```python
import os
import boto3
import math

# grab environment variables
ENDPOINT_NAME = os.environ['ENDPOINT_NAME']
runtime = boto3.client('runtime.sagemaker')

def lambda_handler(event, context):
    
    
    payload = event
    response = runtime.invoke_endpoint(EndpointName=ENDPOINT_NAME, 
                                       ContentType='text/csv', 
                                       Body=payload)
    result = response['Body'].read()
    result = result.decode("utf-8")
    result = result.split(',')
    result = [math.ceil(float(i)) for i in result]
    
    return result[0]
```

<img src="docs/img/sagemaker_images/sm_17.png" width="600">

7. `ENDPOINT_NAME` in the code above is an environment variable that holds the name of the SageMaker model endpoint you deployed using the sample notebook. Scroll down the page and click the button "Manage environment variables" under the "Environment variables" tab. Click "Add environment variable". Call the key "ENDPOINT_NAME" and the value the name of your endpoint - for example, mine was "deploy-tutorial-2020-04-13-16-03-41". Click "Save".

<img src="docs/img/sagemaker_images/sm_18.png" width="600">

### 5.2 Create an API Gateway – Integration request setup

We're almost there, we've created a Python function in Lambda that calls our SageMaker model endpoint. We'll now use API Gateway to help users easily access this function via an API request.

1. Head back to the main AWS dashboard and search for Amazon API Gateway.

<img src="docs/img/sagemaker_images/sm_19.png" width="600">

2. Under the "REST API" tab click "Build".
3. Click the "New API" radio button.
4. Give your API a name like "PredictAbalone" and leave other settings as default. Click "Create API".
5. Next, select "Create Resource" from the "Actions" drop-down menu and give the resource a name like “predictabalone” and click "Create resource".

<img src="docs/img/sagemaker_images/sm_20.png" width="600">

6. From the "Actions" drop-down menu, choose "Create Method" and select "POST".
7. On the screen that appears, choose "Integration type: Lambda Function" and in the "Lambda function" text box, search for and find the function we created earlier (mine was called "predict_abalone"). Click "Save".

<img src="docs/img/sagemaker_images/sm_21.png" width="600">

8. From the "Actions" drop-down menu, choose "Deploy API". Create a new "Deploy stage" called "Test" and then click "Deploy".

<img src="docs/img/sagemaker_images/sm_22.png" width="600">

9. Note the invoke URL that was created when you deployed your API. It should be something like: `https://{restapi_id}.execute-api.{region}.amazonaws.com/test/predictabalone`. Note that the resource name "predictabalone" is not appended automatically.

For more detailed information on how to create an API with API Gateway, refer to the [documentation](https://docs.aws.amazon.com/apigateway/latest/developerguide/how-to-create-api.html). 

### 5.3 Testing our model with Postman

Now that we have a deployed model endpoint and have set up our API we can test everything out. We'll use Postman to send an API request and (hopefully) receive back a prediction from our model. You can download the latest version of Postman [here](https://www.postman.com/downloads/).

1. Open up the Postman application and then select "Create a request".
2. Choose "POST" as the request method and the paste your invoke URL from the previous step into "Enter request url" field.
3. Select the "Body" tab and then the "raw" radio button. Post the following example data into the body field: 

`"0.41,0.325,0.1,0.3555,0.146,0.072,0.105,0,1"`

4. Finally, click "Send" and you should receive a results of `10` back - the predicted age of the abalone for the sent data.

<img src="docs/img/sagemaker_images/sm_23.png" width="600">

# 6. End and next steps<a id=6></a>

Congratulations! You just created a model endpoint deployed and hosted by Amazon SageMaker and were able to invoke that endpoint with the help of API Gateway and a Lambda function - so cool! Have fun integrating this endpoint into your other software/workflows/apps!

We only scratched the surface of Amazon SageMaker's capabilities in this tutorial. I highly recommend checking out the [SageMaker docs](https://docs.aws.amazon.com/sagemaker/latest/dg/whatis.html) to see what else it can do!